In [8]:
!pip3 install python-statemachine
%run helpers.ipynb
%run math_utils.ipynb

In [9]:
from statemachine import StateMachine, State
import numpy as np

In [24]:
class ADCS(StateMachine):
    """
    This class functions as the ADCS Controller.
    
    inputs = []
    outputs = []
    states = []
    
    transitions
    -----------
    
    """
    
    off = State(initial=True)
    detumbling = State()
    pointing = State()
    error = State()
    
    on_initialization = off.to(detumbling)
    to_pointing = detumbling.to(pointing)
    continuous_pointing = pointing.to(pointing)
    
    to_error = pointing.to(error)
    
    def __init__(self):
        self.moment = None
        self.pointing_vec = [0, 0, 0]
        super(ADCS, self).__init__()
        
    def b_dot(self, mag_field_vector, cang):
        detb = np.sqrt(np.dot(mag_field_vector, mag_field_vector))
        m = ((-1) / detb) * np.cross(mag_field_vector, cang)    
        return m

    def get_attitude_err(self, desired, estimated):

        delta = np.matmul(estimated, np.transpose(desired))
        attitude_err = -0.5 * np.array([
            delta[2, 1] - delta[1, 2],
            delta[0, 2] - delta[2, 0],
            delta[1, 0] - delta[0, 1],
        ])

        return attitude_err

    def get_pointing_torque(self, desired_pointing, actual_pointing):

        attitude_err = self.get_attitude_err(desired_pointing, actual_pointing)
        u = -np.matmul(np.diag([.1, .1, .1]), attitude_err)
        J = inertia(6, 0.34, 0.1, 0.1)
        return np.matmul(J, u)


    def get_pointing_moment(self, magnetic_field, desired, actual):
        moment = 1/(np.dot(magnetic_field, magnetic_field)) * np.matmul(skew_symmetric(magnetic_field), self.get_pointing_torque(desired, actual))
        return moment
    
    def detumble(self, mag_field, cang):
        
        # slow down rotation
        self.moment = self.b_dot(mag_field, cang)
        
    def point(self, mag_field, desired, actual):
        
        # update pointing vector
        
        self.moment = self.get_pointing_moment(mag_field, desired, actual)
        c1 = self.moment[0]
        c2 = self.moment[1]
        c3 = self.moment[2]
        
    
    def get_moment(self):
        return self.moment
    

class SafetyMonitor(StateMachine):
    
    safe = State(initial=True)
    error = State()
    
    to_error = safe.to(error, cond="moment_error")
    
    def moment_error(moment):
        return moment > 0.12
    
    def get_state(self):
        return self.current_state.id